In [1]:
!wget https://github.com/PokeMate/name-generator/raw/master/api/static/model.h5 -O model.h5
!wget https://raw.githubusercontent.com/PokeMate/name-generator/master/api/static/names.txt -O names.txt

--2022-01-16 09:37:31--  https://github.com/PokeMate/name-generator/raw/master/api/static/model.h5
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PokeMate/name-generator/master/api/static/model.h5 [following]
--2022-01-16 09:37:32--  https://raw.githubusercontent.com/PokeMate/name-generator/master/api/static/model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221016 (216K) [application/octet-stream]
Saving to: ‘model.h5’

model.h5            100%[===================>] 215.84K  --.-KB/s    in 0.03s   

2022-01-16 09:37:32 (6.82 MB/s) - ‘model.h5’ saved [221016/221016]

--2022-01-16 09:37:32

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [1]:
!pip show h5py

Name: h5py
Version: 2.10.0
Summary: Read and write HDF5 files from Python
Home-page: http://www.h5py.org
Author: Andrew Collette
Author-email: andrew.collette@gmail.com
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: six, numpy
Required-by: tensorflow, keras-vis


In [5]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-3.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 2.9 MB 4.2 MB/s 


In [4]:
import pandas as pd
import numpy as np
import keras
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# from tensorflow.keras.optimizers import RMSprop # tf v2
from keras.optimizers import RMSprop  # tf v1
import tensorflow as tf
import random
import os
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [5]:
# To hide warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [6]:
config = tf.compat.v1.ConfigProto(
    device_count={"GPU": 1}, intra_op_parallelism_threads=1, allow_soft_placement=True
)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
session = tf.Session(config=config) # tf v1
keras.backend.set_session(session) # tf v1

In [7]:
def load_names(input_path):
    names = []
    f = open(input_path)
    for name in f:
        name = name.rstrip()
        names.append(name)
    f.close()

    return names

In [8]:
class NameGenerator:
    def __init__(self, model_path, input_names_path):
        # Load names from the file
        self.input_names = load_names(input_names_path)

        # Make it them to a long string
        self.concat_names = "\n".join(self.input_names).lower()

        # Find all unique characters by using set()
        self.chars = sorted(list(set(self.concat_names)))

        # Build translation dictionaries, 'a' -> 0, 0 -> 'a'
        self.char2idx = dict((c, i) for i, c in enumerate(self.chars))
        self.idx2char = dict((i, c) for i, c in enumerate(self.chars))

        # Count the number of unique characters
        self.num_chars = len(self.chars)

        # Use longest name length as our sequence window
        self.max_sequence_length = max([len(name) for name in self.input_names])

        self.sequence = self.concat_names[-(self.max_sequence_length - 1) :] + "\n"

        # Load the model
        self.model = load_model(model_path)
        self.model._make_predict_function()

    def generate_names(self, gen_amount):
        new_names = []

        # if gen_amount < 1 or gen_amount > 10:
        #     raise (Exception)

        while len(new_names) < gen_amount:

            # Vectorize sequence for prediction
            x = np.zeros((1, self.max_sequence_length, self.num_chars))
            for i, char in enumerate(self.sequence):
                x[0, i, self.char2idx[char]] = 1

            # Sample next char from predicted probabilities
            try:
                with session.as_default():
                    with session.graph.as_default():
                        probs = self.model.predict(x, verbose=0)[0]
                        probs /= probs.sum()
                        next_idx = np.random.choice(len(probs), p=probs)
                        next_char = self.idx2char[next_idx]
                        self.sequence = self.sequence[1:] + next_char

                        # New line means we have a new name
                        if next_char == "\n":

                            gen_name = [name for name in self.sequence.split("\n")][1]

                            # Never start name with two identical chars, could probably also
                            if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
                                gen_name = gen_name[1:]

                            # Discard all names that are too short
                            if len(gen_name) > 2:

                                # Only allow new and unique names
                                if gen_name not in self.input_names + new_names:
                                    new_names.append(gen_name.capitalize())
            except:
                print("Name Generation Error")

        return new_names

In [9]:
model_path = os.path.realpath("./model.h5")
input_names_path = os.path.realpath("./names.txt")
generator = NameGenerator(model_path, input_names_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
amount = 10000
names = generator.generate_names(amount)

In [18]:
with open('aimon_names.txt', 'w') as f:
    f.writelines('\n'.join(names))